The Santa Barbara scene, taken on the years 2013 and 2014 with the AVIRIS sensor over the Santa Barbara region (California) whose spatial dimensions are 984 x 740 pixels and includes 224 spectral bands.

- barbara_2013.mat: 		original image.			740x984x224 

- barbara_2014.mat:		changed image.			740x984x224 

- barbara_gtChanges.mat:	reference data of changes.	740x984x1 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from scipy.io import loadmat
barbara= loadmat('barbara_2013.mat')

In [ ]:
barbara.keys()

In [ ]:
barbara = barbara['HypeRvieW']
barbara.shape

In [ ]:
type(barbara)

In [ ]:
#barbara[:,:,0].shape表示对barbara数组进行索引，取出所有行和列，并且只保留通道为0的那一层。.shape是用来获取数组的形状的属性。
#具体地，barbara[:,:,0]将返回一个新的数组，它只包含barbara数组的第一个通道的所有元素。然后，.shape将返回这个新数组的形状。
barbara[:,:,0].shape

In [ ]:
#找到数组中的最大值
barbara.max()

In [ ]:
barbara.min()

In [ ]:
barbara

In [ ]:
barbara_gt = loadmat('barbara_gtChanges.mat')
print(barbara_gt.keys())

In [ ]:
barbara_gt = barbara_gt['HypeRvieW']
barbara_gt.shape

In [ ]:
barbara_gt

In [ ]:
np.unique(barbara_gt)

In [ ]:
df_barbara_gt = pd.DataFrame(barbara_gt)
df_barbara_gt

In [ ]:
df_barbara_gt.to_csv("df_barbara_gt.csv", index = False)

In [ ]:
barbara_gt.ravel().shape

# 读取数据

In [ ]:
X = barbara
y = barbara_gt
print(f"X shape: {X.shape}\ny shape: {y.shape}")

In [ ]:
#随机生成波段，并展示该波段的图像
import seaborn as sns
sns.axes_style('whitegrid')
fig = plt.figure(figsize=(12, 6))

for i in range(1, 1+6):
    fig.add_subplot(2, 3, i)
    q = np.random.randint(X.shape[2])
    plt.imshow(X[:, :, q], cmap='jet')
    plt.axis('off')
    plt.title(f'band - {q}')

In [ ]:
!pip install plotly
import plotly.express as px

#Plotly是一个用于创建交互式可视化图表的Python库。它提供了多种图表类型和丰富的配置选项，使用户能够创建高质量、可交互的图表和数据可视化。

cls = px.imshow(y, color_continuous_scale='jet')

#jet颜色连续比例尺在数据可视化中常用于表示数值的大小或强度。
#一些常用比例尺：jet viridis plasma inferno magma coolwarm

cls.update_layout(title='Ground Truth', coloraxis_showscale=True)
cls.update_xaxes(showticklabels=False)
cls.update_yaxes(showticklabels=False)
cls.show()

### 将图像转化为CSV存储

In [ ]:
def extract_pixels(X, y, save_name='barbara_all'):
    q = X.reshape(-1, X.shape[2])
    df = pd.DataFrame(q)
    df = pd.concat([df, pd.DataFrame(y.ravel())], axis=1)
    df.columns= [f'band{i}' for i in range(1, 1+X.shape[2])]+['class']
    df.to_csv(f'dataset/{save_name}.csv', index=False)
    
    return df

#将X数组重新整形为一个二维数组，其中第一维表示像素点的数量，第二维表示每个像素点的波段值。

In [ ]:
df = extract_pixels(X, y, save_name='barbara_all')
df
#提取的像素数据会被赋值给变量 df，成为一个DataFrame对象

In [ ]:
df.info()
#打印出DataFrame的基本信息，包括每列的名称、数据类型和非空值数量等

In [ ]:
df.iloc[:, :-1].describe()
#对DataFrame对象 df 进行切片操作，选择除了最后一列之外的所有列。
#.describe() 是DataFrame对象的一个方法，用于生成关于数据的统计描述信息。

## PCA

由于HSI数据集具有较高的维度，因此难以处理庞大的数据。 因此，使用主成分分析（PCA）将数据的维数缩减为3D，这是一种流行且广泛使用的降维技术。 以下代码用于将数据集的尺寸减少为三个。

In [ ]:
from sklearn.decomposition import PCA

pca_components = 85

pca = PCA(n_components = pca_components)
data = df.iloc[:, :-1].values
dt = pca.fit_transform(data)

ev=pca.explained_variance_ratio_

plt.figure(figsize=(12, 6))
plt.plot(np.cumsum(ev))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

plt.show()

#使用matplotlib库绘制了一个图表，图表的横坐标是主成分的数量，纵坐标是累计解释方差的比例。
#通过plt.plot(np.cumsum(ev))绘制了累计解释方差的比例曲线。
#np.cumsum(ev)是将每个主成分的方差比例进行累积求和。plt.xlabel('Number of components')和plt.ylabel('Cumulative explained variance')设置了横坐标和纵坐标的标签。

In [ ]:
pca_components = 40

pca = PCA(n_components = pca_components)
dt = pca.fit_transform(df.iloc[:, :-1].values)
q = pd.concat([pd.DataFrame(data = dt), pd.DataFrame(data = y.ravel())], axis = 1)
q.columns = [f'PC-{i}' for i in range(1, pca_components+1)]+['class']
q.head()
#这段代码的目的是进行主成分分析，并将降维后的数据与标签数据合并到一个新的DataFrame对象中，并为该对象设置列名。

### 可视化PCA之后的光谱图

In [ ]:
print(q.loc[:, f'PC-{i}'].values.shape)

In [ ]:
fig = plt.figure(figsize = (20, 10))

for i in range(1, 1+8):
    fig.add_subplot(2,4, i)
    plt.imshow(q.loc[:, f'PC-{i}'].values.reshape(984, 740), cmap='nipy_spectral')
#获取名为q的DataFrame,选择q中的所有行和名为PC-i的列,并将其转换为一个二维数组。
    plt.axis('off')
    plt.title(f'Band - {i}')

plt.savefig('IP_PCA_Bands.png')

#这段代码使用Matplotlib库创建一个包含8个子图的图表，每个子图显示一个不同的波段图像。
#图表的大小设置为20x10英寸。代码将每个子图添加到2x4的网格中，并将对应波段的图像绘制在每个子图中。

In [ ]:
# saving to .csv
q.to_csv('IP_40_PCA.csv', index=False)

## SVM

In [ ]:
x = q[q['class'] != 0]
X = x.iloc[:, :-1].values
y = x.loc[:, 'class'].values 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11, stratify=y)
svm = SVC(C=100, kernel='rbf', cache_size=10*1024)
svm.fit(X_train, y_train)
ypred = svm.predict(X_test)

In [ ]:
names = ['changed pixels','unchanged pixels']

In [ ]:
data = confusion_matrix(y_test, ypred)
df_cm = pd.DataFrame(data, columns=np.unique(names), index = np.unique(names))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'

plt.figure(figsize = (5,4))
sns.set(font_scale=1.0)#for label size
sns.heatmap(df_cm, cmap="Greens", annot=True,annot_kws={"size": 16}, fmt='d')

plt.savefig('cmap.png', dpi=300)

In [ ]:
print(classification_report(y_test, ypred, target_names = names))

In [ ]:
# Classification Map
l = []
for i in range(len(q)):
    if q.iloc[i, -1] == 0:
        l.append(0)
    else:
        l.append(svm.predict(q.iloc[i, :-1].values.reshape(1, -1))[0])

In [ ]:
clmap = np.array(l).reshape(984, 740).astype('float')
plt.figure(figsize=(10, 8))
plt.imshow(clmap, cmap='jet')

In [ ]:
# 真实数据
y = barbara_gt

plt.figure(figsize=(10, 8))
plt.imshow(y, cmap='jet')
plt.colorbar()
plt.axis('off')
plt.savefig('IP_cmap.png')
plt.show()

## 可视化

In [ ]:
class_labels = {'1':'changed pixels',
                '2':'unchanged pixels',
               }

In [ ]:
q2 = q[q['class'] != 0]
# 添加真实标签列：将数值标签映射到对应的真实标签
q2['label'] = q2.loc[:, 'class'].apply(lambda x: class_labels[str(x)])

In [ ]:
q2['label'].value_counts()

In [ ]:
q2.head()

In [ ]:
import plotly.express as px
count = q2['class'].value_counts()
bar_fig = px.bar(x=count.index[1:], y=count[1:], labels=class_labels, color=count.index[1:])
bar_fig.update_layout(xaxis = dict(title='Class', 
                                   tickmode='array', 
                                   tickvals=count.index[1:].tolist(), 
                                   tickangle = 45,
                                  ),
                      yaxis = dict(title='count',),
                      showlegend=True
                     )
bar_fig.show()

In [ ]:
q2['label'].value_counts().min()

In [ ]:
# 重采样一部分样本以便可视化
# sampling dataset
sample_size = q2['label'].value_counts().min()
sample = q2.groupby('class').apply(lambda x: x.sample(sample_size))
sample

In [ ]:
fig = px.scatter(sample, x="PC-1", y="PC-2", size="class", color="label",
                 hover_name="label", log_x=True, size_max=12)
fig.show()

In [ ]:
scatter_3d = px.scatter_3d(sample, x="PC-1", y="PC-2", z="PC-3", 
                           color="label", size="class", hover_name="label",symbol="label")
# color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"})
scatter_3d.show()
# py.plot(scatter_3d, filename = 'scatter_3d', auto_open=True)

堆叠面积图适用于可视化“部分-整体”的关系，这有助于展现各分类及总体的发展趋势和相互之间的关系。

层叠面积图上最大的面积代表了所有数据量的总和，是一个整体。各个叠起来的面积表示各个数据量的大小，这些堆叠起来的面积图在表现大数据的总分量的变化情况时格外有用，所以层叠面积图非常适用于对比多变量随时间变化的情况。层叠面积图并不能反映总量的变化，但是可以清晰的反应每个数值所占百分比随时间或类别变化的趋势线，这对于分析自变量是大数据、时变数据、有序数据时各个指标分量占比极为有用。

>ref:https://www.edrawsoft.cn/what-is-area-chart/mianjitu

In [ ]:
area_plt1 = px.area(sample, x="PC-1", y="PC-2", color="label", line_group="label")
area_plt1.show()
# py.plot(area_plt1, filename = 'area_plt1', auto_open=True)

In [ ]:
area_plt2 = px.area(sample, x="PC-1", y="PC-3", color="label", line_group="label")
area_plt2.show()
# py.plot(area_plt2, filename = 'area_plt2', auto_open=True)

In [ ]:
area_plt3 = px.area(sample, x="PC-2", y="PC-3", color="label", line_group="label")
area_plt3.show()
# py.plot(area_plt3, filename = 'area_plt2', auto_open=True)

In [ ]:
pair = px.scatter_matrix(sample, dimensions=["PC-1", "PC-2", "PC-3"], color="label")
pair.show()
# py.plot(pair, filename = 'pair_plot_pc', auto_open=True)

In [ ]:
np.linspace(-100, 100, 20, dtype=int).shape

In [ ]:
x = np.linspace(-100, 100, 20, dtype=int)
y = 1.0/(1 + np.exp(-x))

In [ ]:
plt.plot(x,y)